In [1]:
import mysql.connector
import pandas as pd

In [2]:
mydb = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    passwd = '123456',
    database = 'retrowheels')

In [3]:
def query(query):
    kursor = mydb.cursor()
    kursor.execute(query)
    hasil = kursor.fetchall()
    data = pd.DataFrame(data = hasil, columns = kursor.column_names)
    return data

In [4]:
query("show tables")

,Tables_in_retrowheels
0,customers
1,employees
2,offices
3,orderdetails
4,orders
5,payments
6,productlines
7,products


## jumlah total customer yang Anda layani, beserta jumlah total kota & negara asal customer-customer

In [5]:
query("select count(distinct (customerNumber)) as Customers,count(distinct (city)) as Cities,count(distinct (country)) as Countries from customers")

,Customers,Cities,Countries
0,122,95,27


## jumlah karyawan, jumlah kantor & lokasi negaranya, jumlah barang yang dijual, total stok barang & jumlah vendor yang menjadi partner

In [6]:
query('''select count(distinct (e.employeeNumber)) as employee, 
count(distinct (o.officeCode)) as Office, 
count(distinct (o.country)) as Country,
count(distinct (p.productCode)) as Products,
sum(p.quantityInStock) as StockProducts,
count(distinct (p.productVendor)) as Vendors
 from employees e, offices o, products p''')

,employee,Office,Country,Products,StockProducts,Vendors
0,23,7,5,110,89376091,13


## Dari soal sebelumnya tercatat Anda memiliki 110 model die cast dengan total stok 555131 item. Jika dikategorikan, produk yang Anda jual terbagi menjadi 7 product line die cast, yakni model mobil klasik, mobil vintage, sepeda motor, pesawat terbang, kapal laut, kereta api serta truk & bus. Rumuskan single query yang dapat menampilkan harga produk terendah & tertinggi untuk masing-masing kategori

In [7]:
query('select distinct(productLine), min(buyPrice) as minPrice, max(buyPrice) as maxPrice from products group by productLine')

,productLine,minPrice,maxPrice
0,Classic Cars,15.91,103.42
1,Motorcycles,24.14,91.02
2,Planes,29.34,77.27
3,Ships,33.30,82.34
4,Trains,26.72,67.56
5,Trucks and Buses,24.92,84.76
6,Vintage Cars,20.61,86.70


## Rumuskan single query yang dapat menampilkan daftar 10 customer paling royal (paling banyak mendatangkan uang bagi kita), yang total nominal transaksinya paling tinggi. Data yang ditampilkan adalah nama customer, kota & negara asal, beserta total uang yang dihabiskan di produk kita

In [8]:
query('''select c.customerName, c.city, c.country, sum(p.amount) as total from customers c, payments p where c.customerNumber = p.customerNumber
 group by c.customerName order by total desc limit 10''')

,customerName,city,country,total
0,Euro+ Shopping Channel,Madrid,Spain,715738.98
1,Mini Gifts Distributors Ltd.,San Rafael,USA,584188.24
2,"Australian Collectors, Co.",Melbourne,Australia,180585.07
3,Muscle Machine Inc,NYC,USA,177913.95
4,"Dragon Souveniers, Ltd.",Singapore,Singapore,156251.03
5,"Down Under Souveniers, Inc",Auckland,New Zealand,154622.08
6,"AV Stores, Co.",Manchester,UK,148410.09
7,"Anna's Decorations, Ltd",North Sydney,Australia,137034.22
8,Corporate Gift Ideas Co.,San Francisco,USA,132340.78
9,"Saveley & Henriot, Co.",Lyon,France,130305.35


## Pada 2003-06-05, terdapat pembayaran masuk sebesar US dollar 14571.44. Tampilkan data seputar transaksi tersebut, mencakup nama customer yang melakukan pembayaran, nama produk yang dibeli, jumlah tiap produk yang dibeli dan harga satuannya. Pastikan total harga yang dibeli sesuai dengan data pembayaran masuk

In [9]:
query(''' select c.customerName, p.productName, od.quantityOrdered, od.priceEach
 from customers c, orderdetails od, orders o, products p, payments pa
 where od.orderNumber = o.orderNumber and c.customerNumber = o.customerNumber and od.productCode = p.productCode and pa.paymentDate = '2003-06-05' limit 4''')

,customerName,productName,quantityOrdered,priceEach
0,Atelier graphique,1965 Aston Martin DB5,26,120.71
1,Atelier graphique,1999 Indy 500 Monte Carlo SS,46,114.84
2,Atelier graphique,1948 Porsche Type 356 Roadster,34,117.26
3,Atelier graphique,1966 Shelby Cobra 427 S/C,50,43.27
